<a href="https://colab.research.google.com/github/mrchapagain/ConsumerDataAnalytics/blob/main/Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here i try to explore different dataset to look if they can be connected, linked or somehow can talk to eachother.**
*   *Data by Frederik*
*   *Frida dataset of Nutrient content of food items*
*   *Reciept from my personal Storebox account*
*   *link from TradeSync data from GS1*
*   *Climate Data from Concito*

In [5]:
#!pip install PyPDF2

In [40]:
# Necessary libaries
import pandas as pd
import os
import PyPDF2

# Data by Frederik

***Columns description***

*   **itemname:** name provided by the merchant. It often, but not always, contains a enough information to ID the product.  Sometimes weight, or other information is also provided here.
*   **itemnumber:** the product itemnumber. It can be a standardized GTIN 13 number ( 12-13 digits) supported by GS1 or a non standard number created by the retailer to identify i.e. Bananas
*   **category:** a category code not sure how to interpret and often missing
*   **percent_purchases:** Percent off all receipt lines in the dataset where each line is a purchas

In [86]:
# Import data
path_iss_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/cpd_summary.csv"

# Open data as csv file
data_iss= pd.read_csv(path_iss_data, sep= ";", index_col=False, encoding='latin-1')

# lets see frist 3 and last 3 rows od the dataset
data_iss.head(3).append(data_iss.tail(5))

,itemname,itemnumber,category,percent_purchases
0,PANT,8880171,845,"3,369"
1,PANT,8880171,NaN,"2,055"
2,PANT,8880172,845,"1,497"
112761,PINOTEX SUPERDEC SOR,61009053404,6100,"-0,002"
112762,QUICK GEVINSTER,214196,926,"-0,002"
112763,UKRUDTSDUG BASIC 1X,42139019916,4213,"-0,002"
112764,TRAPPEVANGE,5708832546219,600,"-0,004"
112765,BR MÆRKER RETUR,5766183013538,21,"-0,005"


In [125]:
# Only print one selective row of interest to deep deeper
data_iss[data_iss.itemname == "ØKO BANANER, LØSE"] # or "ØKO BANANER, LØSE", HAVFRISK LAK, APPELSINER 2 KG,Agurk

,itemname,itemnumber,category,percent_purchases
11,"ØKO BANANER, LØSE",5712873336944,89,"0,567"
9212,"ØKO BANANER, LØSE",5712580050171,89,"0,002"
27542,"ØKO BANANER, LØSE",20008932,89,"0,001"
27543,"ØKO BANANER, LØSE",20014018,89,"0,001"
27544,"ØKO BANANER, LØSE",20023393,89,"0,001"
27545,"ØKO BANANER, LØSE",5712580000114,89,"0,001"
27546,"ØKO BANANER, LØSE",5712580024349,89,"0,001"
27547,"ØKO BANANER, LØSE",5712580034065,89,"0,001"
27548,"ØKO BANANER, LØSE",5712580251912,89,"0,001"
27549,"ØKO BANANER, LØSE",5712580317717,89,"0,001"


***In above ddataset, only one food itemname (fx `"ØKO BANANER, LØSE"`) is appear in different rows with;***
*   Many different itemnumber, even some are 13 digits and other are 8 digits
*   NULL percent purchage

In [58]:
data_iss.itemnumber.unique()

array(['8880171', '8880172', '4', ..., '4006676002924', '61009053404',
       '5708832546219'], dtype=object)

# FRIDA Food Composition Database
***FRIDA Food Composition Database  contain mainly Nutrition value of the each food items***

*   ***Category list:*** https://frida.fooddata.dk/food/lists/grouped?lang=en
*   ***Data display inspiration:*** https://foodb.ca/foods/FOOD00011
*   ***Display on FRIDA:*** https://frida.fooddata.dk/food/754?lang=en

In [44]:
# Import FRIDA Nutrient data
path_frida_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Lets open data as they are but only 26 columns out of 201 columns
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=0, skiprows=0).round(decimals = 2).iloc[:, :25]
data_frida.head(3)

,Unnamed: 1,Unnamed: 2,Svind,"Energi, kJ","Energi, kcal",Nitrogen-til-protein faktor,"Nitrogen, total","Protein, videnskabelig","Protein, deklaration","Kulhydrat, difference",...,FCF,Alkohol,Aske,Tørstof,Vand,A-vitamin,Retinol,beta-caroten,D-vitamin,D3-vitamin
Nummer,gruppe,Navn,%,kJ,kcal,-,g,g,g,g,...,-,g,g,g,g,RE,µg,µg,µg,µg
1,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,8.3,...,0.8,0,0.4,10,90,3.33,0,40,0,0
2,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,14.3,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0


In [134]:
# Lets define columns name that are readeable
column_names= ['Gruppe', 'Navn', 'Svind_%', 'Energy_kj', 'Energy_kcal', 'Nitrogen_til_protein_faktor', 'Nitrogen_total_g', 'Protein_videnskabelign_g', 'Protein_deklaration_g', 'Kulhydrat_difference_g', 'Kulhydrat_tilgængelig_g', 'Kulhydrat_deklaration_g', 'Tilsat_sukker_g', 'Kostfibre_g', 'Fedt_total_g', 'FCF', 'Alkohol_g', 'Aske_g', 'Tørstof_g', 'Vand_g', 'A_vitamin_RE', 'Retinol_µg', 'beta_caroten_µg', 'D_vitamin_µg', 'D3_vitamin_µg']

# Lets open with little modificaton so easy to read and understand
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=None, skiprows=2).round(decimals = 2).iloc[:, :25].reset_index(drop=True)
# Assign columns name that defined just above
data_frida.columns= column_names

# lets see frist 3 and last 3 rows od the dataset
data_frida.head(3).append(data_frida.tail(3))
#data_frida[data_frida.Gruppe == '"Frugt"-grøntsager']
data_frida[data_frida.Navn == "Banan, rå"]


,Gruppe,Navn,Svind_%,Energy_kj,Energy_kcal,Nitrogen_til_protein_faktor,Nitrogen_total_g,Protein_videnskabelign_g,Protein_deklaration_g,Kulhydrat_difference_g,...,FCF,Alkohol_g,Aske_g,Tørstof_g,Vand_g,A_vitamin_RE,Retinol_µg,beta_caroten_µg,D_vitamin_µg,D3_vitamin_µg
2,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,22.6,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0


In [92]:
row=data_frida[data_frida.Navn== "Kylling, bryst (filet), kogt, pålæg"]   # Kylling, bryst (filet), kogt, pålæg,  ØKO BANANER, LØSE
print("Id: ", row.index[0])
print("Category: ", row.Gruppe.values[0])
print("Name: ", row.Navn.values[0])
print("Total Energy: ", row.Energy_kj.values[0], "Kj (", row.Energy_kcal.values[0], "Kacl)")
print("Total Energy: ", row.Energy_kcal.values[0])
print("Total Energy: ", row.Energy_kj.values[0])

Id:  561
Category:  Kylling og høne
Name:  Kylling, bryst (filet), kogt, pålæg
Total Energy:  475 Kj ( 113 Kacl)
Total Energy:  113
Total Energy:  475


In [85]:
# To see NAN value and data types of each columns
#data_frida.info()

***Above dataset do not contain NAN value and all of the columns are object type, so some of the columns need to be changed to interger or float data type to work as numbers value***

# Recipt from my personal Storebox account
- This data set i download from my own storebox account. Lets see what kind of information we can extract from storebox account

***The Store-box dataset contain one dataset of recieptand another dataset of the actual list of itemspurchaged each time***

In [100]:
#Import data
path_storebox_data= "https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/storbox_recipts.json"

#Loading dat in dataframe 
data_storebox= pd.read_json(path_storebox_data)
# ['id', 'type', 'receiptId', 'purchaseDate', 'purchaseDateTimeString', 'orderNumber', 'price', 'merchant', 'receiptLines', 'terminalId', 'payments', 'cards', 
#     'barcode', 'headerTexts', 'footerTexts', 'userIds', 'imageId', 'receiptImage', 'note', 'tags', 'headerText', 'footerText', 'purchaseDateTimeZoneOffset', 'legalReceipt']

# Lets open only columns that make sence
data_storebox= data_storebox[["id", "receiptId", "purchaseDate", "orderNumber", "price", "merchant",'barcode']]
data_storebox.head(3)

,id,receiptId,purchaseDate,orderNumber,price,merchant,barcode
0,236769820,05jrqpia3ofdzsbq773yme4on0ko1f87,1520106179000,5320020898,"{'amount': 229.29, 'vat': 45.86, 'currency': '...","{'merchantId': 'netto', 'storeId': None, 'name...","{'type': 'interleaved2of5', 'value': '07344180..."
1,323704901,0dtsbqbwkirnyxfjqwdaogf5bli877yi,1546781345000,8190010157,"{'amount': 19.15, 'vat': 3.83, 'currency': 'DK...","{'merchantId': 'netto', 'storeId': '7345', 'na...","{'type': 'interleaved2of5', 'value': '07345190..."
2,230788984,0of79b19he9g7g5wazomg1tdoc3yh0xk,1518030461000,5080021069,"{'amount': 115.22, 'vat': 23.04, 'currency': '...","{'merchantId': 'netto', 'storeId': None, 'name...","{'type': 'interleaved2of5', 'value': '07344180..."


***With the receiptId i can go to indiviisual reciept and extract list of purchage items. However the reciept is in PDF so need little advance RegEx code so i have not done it yet but will update soon.***


**Now lets  open reciept of  of the very first column (receiptId= "05jrqpia3ofdzsbq773yme4on0ko1f87") of above data**

In [81]:
# Lets open reciept of very first columns (receiptId= "05jrqpia3ofdzsbq773yme4on0ko1f87") of above dataset (data_storbox)
data_storbox_reciept= "/content/05jrqpia3ofdzsbq773yme4on0ko1f87-14736470631022876123.pdf" # "https://github.com/mrchapagain/food_ontology/raw/main/storbox_recipt_pdf.pdf"

# read information from PDF file (open it in read binary mode)
pdfFileObj = open(data_storbox_reciept, 'rb')

# call and store PdfFileReader
# object in pdfReader
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# to access first page if the PDF file have multiple pages, pass 0
pageObj = pdfReader.getPage(0)

# extract the page object by extractText() function
texts = pageObj.extractText()

# print the extracted texts
print(texts)

UTTERSLEVVEJ 11
2400 Kbh. NV
SØDMÆLK 8,50
SØDMÆLK 8,50
OLLINEO SOLSIKKE OL 9,95
CRYSLI 4 NØD 32,85
DANONINO 14,50
BACON I SKV. 2-PAK 17,90
SPINAT 12,09
5 BANANER,LØSE
A 2,00 10,00
RABAT 5,00-
RØDLØG 8,00
UDL. AGURK 5,00
HAVFRISK LAK 89,00
RABAT 20,00-
5 PÆRER, LØSE
A 2,00 10,00
APPELSINER 2 KG 18,00
GRANATÆBLE, LØS 10,00
TOTAL 229,29
KONTANT 31,50
TOTAL 197,79
DANKORT 197,79
MOMS UDGØR 45,86
Du blev betjent af:
Kevin
9 2 898 03 03 18 20:44
Butik 7344 MOMSNR.35954716
KIG FORBI WWW.NETTO.DK
OG WWW.JOB.NETTO.DK
8-22 ALLE UGENS 7 DAGE
2018-03-03 20:44
Køb DKK 197,79
_________
Dankort PSN: 00
Contactless
XXXX XXXX XXXX 4955
Term: 80471832-143669
NETS A/S
4815734
KC1 Nets no:0004815734
ATC:00292 AED:000000
AID: A0000001214711
PSAM: 5374978-0000544044
ARC:00 STATUS:0000


***I have extracted data from PDF file now first need to turn it to dictionary and then DataFrame.***

In [82]:
lines= texts.rstrip("\n")
lines

'UTTERSLEVVEJ 11\n2400 Kbh. NV\nSØDMÆLK 8,50\nSØDMÆLK 8,50\nOLLINEO SOLSIKKE OL 9,95\nCRYSLI 4 NØD 32,85\nDANONINO 14,50\nBACON I SKV. 2-PAK 17,90\nSPINAT 12,09\n5 BANANER,LØSE\nA 2,00 10,00\nRABAT 5,00-\nRØDLØG 8,00\nUDL. AGURK 5,00\nHAVFRISK LAK 89,00\nRABAT 20,00-\n5 PÆRER, LØSE\nA 2,00 10,00\nAPPELSINER 2 KG 18,00\nGRANATÆBLE, LØS 10,00\nTOTAL 229,29\nKONTANT 31,50\nTOTAL 197,79\nDANKORT 197,79\nMOMS UDGØR 45,86\nDu blev betjent af:\nKevin\n9 2 898 03 03 18 20:44\nButik 7344 MOMSNR.35954716\nKIG FORBI WWW.NETTO.DK\nOG WWW.JOB.NETTO.DK\n8-22 ALLE UGENS 7 DAGE\n2018-03-03 20:44\nKøb DKK 197,79\n_________\nDankort PSN: 00\nContactless\nXXXX XXXX XXXX 4955\nTerm: 80471832-143669\nNETS A/S\n4815734\nKC1 Nets no:0004815734\nATC:00292 AED:000000\nAID: A0000001214711\nPSAM: 5374978-0000544044\nARC:00 STATUS:0000'

# Climate Data from Concito

In [ ]:
#Import data
path_climate_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/ClimateData.xlsx"

# Loading the data into the data-frame
col_names= ["id", "Product_dk", "Category_dk", "Product_en", "Category_en", "Unit", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq/kg", "Energy_KJ", "Fat_g", "Carb_g", "Protein_g", "Data_Source", "Comments", "GPC_Level4_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_Category_en", "GPC_Category_dk", "GPC_Level4_dk", "product_type", "GPC_level1", "Food_group", "GPC_level2", "Un/Processed", "GPC_Level3", "Extra_category"]

# Original columns name are replaced with ralavant names (col_names)
data_climate= pd.read_excel(path_climate_data, sheet_name=1, index_col=False, header=None, skiprows=1, names= col_names).round(decimals = 2)
data_climate.head(3)

***The reason to open this climate dataset is to see if it is possible link other dataset with column name "GPC_Level" which is releted to GS1 Trade Sync code.***

**Let´s have a look dataset with only selected columns that might make sense\**

In [ ]:
cols_to_use= ["Product_en", "Category_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_level1", "GPC_level2", "GPC_Level3", "GPC_Level4_dk", "Fat_g", "Carb_g", "Protein_g", "Energy_KJ", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq/kg"]

data_climate_selected= data_climate.loc[:, cols_to_use].round(decimals = 2)
data_climate_selected.head()

# Data from COOP store (store data)

In [78]:
path_coop_data= "https://raw.githubusercontent.com/mrchapagain/ConsumerDataAnalytics/main/storedata/storedata_csv.csv"
#data_coop= pd.read_excel(path_coop_data, sheet_name=1, header=0, skiprows=0).round(decimals = 2)

data_coop= pd.read_csv(path_coop_data, index_col=0, sep= ";", on_bad_lines='skip', engine='python')

data_coop.head()

""
"PartitionKey,RowKey,Timestamp,ChainId,ChainName,Kardex,StoreId,StoreName,Address,City,Zipcode,Active,Phone,CvrNumber,ValidFrom,MondayFrom,MondayTo,TuesdayFrom,TuesdayTo,WednesdayFrom,WednesdayTo,ThursdayFrom,ThursdayTo,FridayFrom,FridayTo,SaturdayFrom,SaturdayTo,SundayFrom,SundayTo"
"1,20000101,2017-08-11T12:23:38.802Z,3,SuperBrugsen,6598,1,SuperBrugsen Plukbutik (Ringe),""Skovvej 21, Espe"",RINGE,5750,false,26770970,41287519,2000-01-01T00:00:00.000Z,09:00,19:00,09:00,19:00,09:00,19:00,09:00,19:00,09:00,19:00,08:00,16:00,Lukket,Lukket"
"10,20000101,2017-08-11T12:23:37.542Z,3,SuperBrugsen,4155,10,SuperBrugsen Herfølge,Billesborgvej 26,HERFØLGE,4681,true,56274026,26259495,2000-01-01T00:00:00.000Z,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00"
"10,20180405,2018-04-04T13:23:36.870Z,3,SuperBrugsen,4155,10,SuperBrugsen Herfølge,Billesborgvej 26,HERFØLGE,4681,true,56274026,26259495,2018-04-05T00:00:00.000Z,08:00,20:45,08:00,20:45,08:00,20:45,08:00,20:45,08:00,20:45,08:00,20:45,08:00,20:45"
"100,20000101,2017-08-11T12:23:43.921Z,2,Kvickly,6160,100,Kvickly Svendborg City,Gerritsgade 33,SVENDBORG,5700,true,62176600,26259495,2000-01-01T00:00:00.000Z,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00"
"100,20180820,2018-08-20T09:23:37.808Z,2,Kvickly,6160,100,Kvickly Svendborg City,Gerritsgade 33,SVENDBORG,5700,true,62176600,26259495,2018-08-20T00:00:00.000Z,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00"


***These information are saved wrong way; all the columns of csv in one column so it open like one column***

So i am going to open with string split method with comma " ,"

# TradeSync data from GS1 

In [65]:
import os
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

In [74]:
#Import data
path_tradesync_data= "/content/testdata_gs1.xml" # "https://github.com/mrchapagain/food_ontology/blob/main/gs1_data/testdata.xml"

In [75]:
tree = ET.parse(path_tradesync_data)
root = tree.getroot()
print("Roots:", root.tag)
print("Attrib:", root.attrib)
print("Childerns: ", list(root))


Roots: {urn:gs1:gdsn:catalogue_item_notification:xsd:3}catalogueItemNotificationMessage
Attrib: {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'urn:gs1:gdsn:catalogue_item_notification:xsd:3 http://www.gdsregistry.org/3.1/schemas/gs1/gdsn/CatalogueItemNotification.xsd'}
Childerns:  [<Element '{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}StandardBusinessDocumentHeader' at 0x7f8e8d162f50>, <Element 'transaction' at 0x7f8e8cce3590>]


***This part is hidden because the datast is very big and  take somany lines***

In [76]:
# Code to have a look whole data
"""# Reading the data inside the xml file to a variable under the name  data
with open(path_tradesync_data, 'r') as f:
    for line in f.readlines():
      print(line)
"""

# Code to explore tags
"""# The xml file has a LOT of tags!
list_of_items= []
stopAt = 200
for elem in root.iter():
    print(elem.tag)
    list_of_items.append(elem.tag)
    #print(elem.attrib)
    if stopAt==0:
        break
    stopAt -= 1"""

'# The xml file has a LOT of tags!\nlist_of_items= []\nstopAt = 200\nfor elem in root.iter():\n    print(elem.tag)\n    list_of_items.append(elem.tag)\n    #print(elem.attrib)\n    if stopAt==0:\n        break\n    stopAt -= 1'

In [77]:
#print(ET.tostring(root, encoding='utf8').decode('utf8'))
list_of_items_to_find= ["entityIdentification", "brandName", "functionalName", "gln", "gtin", "creationDateTime", "dataRecipient", "contactDescription", "partyAddress", "gpcCategoryCode", "grossWeight", "descriptionShort", "allergenSpecificationName", "ingredientStatement", "tradeItemTemperatureConditionTypeCode", "uniformResourceIdentifier"]
#  "additionalTradeItemIdentification", "partyName",

#Lets start the dictionary to collecct items
dict_item_text = {}

# Loop over the tags that make sense (from list above)
for list_item in list_of_items_to_find:
  
  # Again  looping to get the text value
  for item in root.iter(list_item):
    dict_item_text[list_item] = item.text

# Lets see what we have c collected from the loop
dict_item_text

{'allergenSpecificationName': '2003/89/EC',
 'brandName': 'Gammeldags Flødeis',
 'contactDescription': 'Premier Is, Mejerigaarden A/S',
 'creationDateTime': '2019-11-05T11:35:49.3113569+01:00',
 'dataRecipient': '5790000050000',
 'descriptionShort': 'Karamel ',
 'entityIdentification': 'CatalogueItemNotification-b10a17fc-ef9b-42b1-a0b1-76857c8a26b9',
 'functionalName': 'Fløde is',
 'gln': '5790000701247',
 'gpcCategoryCode': '10000215',
 'grossWeight': '350',
 'gtin': '05766632526800',
 'ingredientStatement': 'Ingredienser: 35 % FLØDE, SKUMMETMÆLK, sukker, glukosesirup, sheaolie, rapsolie, sødet kondenseret SKUMMETMÆLK, SKUMMETMÆLKSPULVER, vand, kokosolie, fedtfattigt kakaopulver, Bourbon vaniljeekstrakt, ekstraherede vaniljekorn, SMØR, palmeolie, majsstivelse, helt hærdet kokosfedt, naturlig vaniljearoma, salt, aroma, paprikaekstrakt, emulgatorer (E 322 (af SOJA), E 471), stabilisatorer (E 466, E 412, E 407, E 440), surhedsregulerende midler (E 331, E 334).\xa0 Kan indeholde spor af: 

***Above dictinary are the information extracted from GS1 xml file associated with each item from the purchage list***

*We can grasp a lot of infoation about the item*